<a href="https://colab.research.google.com/github/Bennath-coder/Bennath-coder/blob/main/De%CC%81couverte_de_Gradio_Live%20coding_vNPz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pour débuter avec Gradio, nous aurons besoin d'installer la librairie dédiée.

In [1]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 2.8 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
tor

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import pickle
import gradio as gr
from sklearn.feature_extraction.text import CountVectorizer
from keras.models import load_model

Notre objectif est de venir créer une interface graphique avec Gradio afin d'intéragir avec nos modèles comme on pourrait le faire avec une application en production.

Gradio va nous permettre de créer et de personnaliser cette interface.

Nous travaillerons à partir du dataset *SMS Spam Collection*. Ce jeu de données est une collection de SMS pouvant être des spams.

On cherche à créer une application simple sur laquelle l'utilisateur a le choix du modèle qu'il utilise et peut venir rentrer son propre sms afin de tester s'il s'agit d'un spam ou non.

## Chargement des données

On commence par récupérer les données et les mettre sous la forme d'un dataset pandas.

In [9]:
with open(f'SMSSpamCollection.txt') as f: # On lit le fichier
  data = f.readlines()

data = [content.split('\t') for content in data] # On sépare les mails

y = []
for target in [content[0] for content in data]: # On crée la colonne y
  y.append(0 if target=='ham' else 1)
y = np.array(y)

X = [content[1].strip('\n') for content in data] # On crée la colonne X
X = np.array(X)

df = pd.DataFrame([X, y]).T # Puis le dataframe
df.columns = ['sms_content', 'is_spam']
df['is_spam'] = df['is_spam'].astype('int')

In [10]:
df.head()

,sms_content,is_spam
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0


In [11]:
df.shape

(5574, 2)

Nous allons utiliser un **vectorizer** qui va nous permettre de transformer les données textuelles sous forme de vecteurs compréhensibles par nos modèles.

In [12]:
from sklearn.model_selection import train_test_split

X = df['sms_content'].values
y = df['is_spam'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
vectorizer = CountVectorizer()
vectorizer.fit(X_train)

CountVectorizer()

## Chargement des modèles

Nous allons récupérer deux modèles déjà entraînés sur le problème de classification des SMS.

Importons d'abord un modèle de régression logistique :

In [14]:
with open('logistic_regression.pkl', 'rb') as f:
    logistic_regression = pickle.load(f)

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.2.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [15]:
logistic_regression

LogisticRegression(multi_class='auto')

Puis un réseau de neurones Keras :

In [16]:
neural_network = load_model('neural_network.h5')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [17]:
neural_network.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_2 (Dense)                      │ (None, 5)                   │          38,635 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 5)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │               6 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 38,643 (150.95 KB)

 Trainable params: 38,641 (150.94 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

A partir de ces deux modèles, il faut que l'on puisse réaliser une prédiction à partir d'un nouveau SMS. Ecrivons une fonction pour cela :

In [19]:
def prediction(model_choice, sms_to_test):
  # On transforme le SMS que l'on souhaite tester en vecteur
  sms_to_test = pd.Series(sms_to_test)
  sms_to_test = vectorizer.transform(sms_to_test)

  # Puis on choisit un modèle parmi les deux disponibles.
  if model_choice == 'Neural Network':
    prediction = neural_network.predict(sms_to_test)[0][0]
    confidence = f" (le modèle est confiant à {prediction:.2%})"
    return "Le SMS n'est pas un spam" + confidence if prediction < 0.5 else "Le SMS est un spam" + confidence

  elif model_choice == 'Logistic Regression':
    prediction = logistic_regression.predict_proba(sms_to_test)[0][1]
    confidence = f" (le modèle est confiant à {prediction:.2%})"
    return "Le SMS n'est pas un spam" + confidence if prediction < 0.5 else "Le SMS est un spam" + confidence

## Interface Gradio

Avec très peu de code, on peut construire une application Gradio fonctionnelle qui répond à nos besoins:

In [ ]:
# Créer une interface Gradio
demo = gr.Interface(fn=prediction, # Il faut passer ici la fonction que l'on a créé
                    inputs= [gr.Radio(['Neural Network', 'Logistic Regression'], label="Model"), #Les inputs décrivent le fonctionnement de l'app
                             gr.Textbox()],
                    outputs="text")

demo.launch(debug=True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://9a76e4b1f35f797691.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 326ms/step


Quelques SMS à tester :

> You just win 100 dollars ! You have won a week in mexico ! Please contact me to collect your gift and reward

> You have won a 1 week free membership in our 1 000 000 prize jackpot !

> Thanks for your subscription to our service, your mobile will be charged $5/month







